# Week 4 simulations for dataset construction

In [ ]:
## Install needed packages
# skip this if you have them already
#!pip install eppy esoreader tqdm shutil

In [ ]:
## import needed packages
# No need to modify anything here

import pandas as pd
import numpy as np
from eppy.modeleditor import IDF
from eppy import modeleditor
import esoreader
import os
import sys
from itertools import product
from tqdm import trange
import shutil

def clear_output():
    """
    clear output for both jupyter notebook and the console
    """
    os.system('cls' if os.name == 'nt' else 'clear')
    if 'ipykernel' in sys.modules:
        from IPython.display import clear_output as clear
        clear()

In [ ]:
## define the ESO class to read the .eso file
# just run this cell once 
# No need to modify anything here

class ESO:
    def __init__(self, path):
        self.dd, self.data = esoreader.read(path)
        
    def read_var(self, variable, frequency = "Hourly"):
        return [
            {"key": k,
             "series": self.data[self.dd.index[frequency, k, variable]]}
            for _f, k, _v in self.dd.find_variable(variable)
        ]
        
    def get_df(self, variable, frequency = "Hourly"):
        dic = self.read_var(variable, frequency)
        key = [each["key"] for each in dic]
        values = [each["series"] for each in dic]
        df = pd.DataFrame(values,index = key).T
        return df
    
    def total_kwh(self, variable, frequency = "Hourly"):
        j_per_kwh = 3_600_000
        results = self.read_var(variable,frequency)
        return sum(sum(s["series"]) for s in results)/j_per_kwh

In [ ]:
## Set up EnergyPlus IDD file
# need to change the path to your EnergyPlus root directory

eplus_root = r"C:\EnergyPlusV25-1-0" # Change this to your EnergyPlus root directory
iddfile = os.path.join(eplus_root,"Energy+.idd")

try:
    IDF.setiddname(iddfile)
except modeleditor.IDDAlreadySetError as e:
    pass

root_dir = os.getcwd()
print("Current working directory:", root_dir)

## Run simulation

In [ ]:
## read in the base idf file
# complete the code to define the simulation parameter space

idf = IDF(os.path.join(root_dir,r"model_week4.idf"))

## Define mapping dictionaries
MAP_WEATHER = {
    "San Francisco":os.path.join(root_dir,r"..\weather_data\USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw"),
    "Sacramento":os.path.join(root_dir,r"..\weather_data\USA_CA_Sacramento.Intl.AP.724930_TMY3.epw"),
    "Chicago":..., # filled it with your path
    "New York":... # filled it with your path
}

MAP_WWR = {
    0.25: [2,3.75], 
    0.40: [2,6],
    0.60: [0.5,9], #(10*3)*0.6/2
}

## Define parameter space
Location = ["San Francisco", "Sacramento", "Chicago", "New York"]
Wall_Rvalue = [14.3, 21.9, 29.7]  # in ft2Fhr/Btu
WWR = [0.25, 0.40, 0.60] 
SHGC = ... # filled it with the SHGC values

## Generate simulation parameter setting dataframe and store it
keys = ["Location","Wall_Rvalue","WWR","SHGC"]
dic = [dict(zip(keys, v)) for v in product(Location,Wall_Rvalue,WWR,SHGC)]
df = pd.DataFrame(dic)
# format the Wall_Rvalue to m2K/W
df["Wall_Rvalue"] = ...  # filled it to make the wall R value convert to m2K/W 

# store simulation parameter settings
df.to_csv(os.path.join(root_dir,"week4_all_parameters.csv"))

In [ ]:
## Modify idf function
# complete the function to modify the idf based on the modify_dict
def modify_idf(idf,modify_dict):
    # modify wall R-value
    ...

    # modify WWR
    window = idf.idfobjects["WINDOW"]
    new_window_x, new_window_length = MAP_WWR[modify_dict["WWR"]]
    window[0]["Starting_X_Coordinate"] = new_window_x
    window[0]["Length"] = new_window_length

    # modify SHGC
    ...

    # modify weather file
    ...
    return idf

In [ ]:
## Run simulations
# complete the code to run simulations and store the results
# try to fill it in following steps:
# 1. define an empty list/dataframe to store results
# 2. loop through each row in the simulation parameter dataframe
# 3. change the output folder name to avoid overwriting
# 4. modify the idf based on the current parameter settings
# 5. run one simulation
# 6. read results and store them in the defined list/dataframe
# 6*. (optional) delete the output folder to save space
# 6*. (optional) clear output for better visualization
# 7. after running all simulations, store the results to a csv file

...

In [ ]:
## Load results and analyze
result_df = ...
df = ...

# complete the analysis code here